In [ ]:
# Initial imports and notebook setup, click arrow to show

import HARK.ConsumptionSaving.ConsIndShockModel as Model        # The consumption-saving micro model
import HARK.SolvingMicroDSOPs.EstimationParameters as Params    # Parameters for the consumer type and the estimation
from HARK.utilities import plotFuncsDer, plotFuncs

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Set up default values for CRRA, DiscFac, and simulation variables in the dictionary 
Params.init_consumer_objects["CRRA"]= 2.00            # Default coefficient of relative risk aversion (rho)
Params.init_consumer_objects["DiscFac"]= 0.97         # Default intertemporal discount factor (beta)
Params.init_consumer_objects["PermGroFacAgg"]= 1.0    # Aggregate permanent income growth factor 
Params.init_consumer_objects["aNrmInitMean"]= -10.0   # Mean of log initial assets 
Params.init_consumer_objects["aNrmInitStd"]= 1.0      # Standard deviation of log initial assets
Params.init_consumer_objects["pLvlInitMean"]= 0.0     # Mean of log initial permanent income 
Params.init_consumer_objects["pLvlInitStd"]= 0.0      # Standard deviation of log initial permanent income

In [ ]:
# Make a lifecycle consumer to be used for estimation
ThisType = Model.IndShockConsumerType(**Params.init_consumer_objects)

In [ ]:
# Solve and simulate the model
ThisType.solve()
ThisType.unpackcFunc()
ThisType.track_vars = ['aNrmNow','pLvlNow', 'aLvlNow','cNrmNow','mNrmNow','TranShkNow'] # Variables to be tracked: 't_cycle'? 'pLvlNow'
ThisType.T_sim = 120
ThisType.initializeSim()
ThisType.simulate()

In [ ]:
# Define the saving rate function
def savingRateFunc(SomeType, m):
    """
    Parameters:
    ----------
        SomeType: 
             Agent type that has been solved and simulated.
        
        
    Returns:
    --------
        SavingRate: float
    
    """
    inc = (SomeType.Rfree -1.)*(m-1.)+1.
    cons = SomeType.solution[0].cFunc(m)
    Saving = inc - cons
    SavingRate = Saving / inc
    return SavingRate  

In [ ]:
# Create a Giant matrix gathering:
# 't_now', 'aNrmNow_hist', 'cNrmNow_hist', employment-status in date t, in date t-1, aLvlGro_hist, Saving rate

w, h = 1, ThisType.T_cycle
giant_list = [[0 for x in range(w)] for y in range(h)]
SavingRate_list = []

for t in range(1,ThisType.T_cycle+1):
    #aLvlGro_hist[0] = 0 # set the first growth rate to 0, since there is no data for period 0
    aLvlGroNow = np.log(ThisType.aNrmNow_hist[t]/ThisType.aNrmNow_hist[t-1]) # (10000,)
    
    # Call the saving rate function with test value for 
    SavingRate = savingRateFunc(ThisType, ThisType.mNrmNow_hist[t] )
      
    SavingRate_list.append(SavingRate)

    # Create elements of matrix list
    matrix_list = [0 for number in range(7)]
    matrix_list[0] = t
    matrix_list[1] = ThisType.aNrmNow_hist[t]
    matrix_list[2] = ThisType.cNrmNow_hist[t]
    matrix_list[3] = ThisType.TranShkNow_hist[t]
    matrix_list[4] = ThisType.TranShkNow_hist[t-1]
    matrix_list[5] = aLvlGroNow
    matrix_list[6] = SavingRate
    
    giant_list[t-1] = matrix_list
    
#print giant_list

In [ ]:
# Plot the consumption functions during working life
print('Consumption functions while working:')
mMin = min([ThisType.solution[t].mNrmMin for t in range(ThisType.T_cycle)])
plotFuncs(ThisType.cFunc[:ThisType.T_retire],mMin,5)

In [ ]:
# Plot Saving rate of one agent over time
print('Saving rate of one agent:')
cycle = []
s = []
for t in range(ThisType.T_cycle):
    cycle.append(t)
    s.append(SavingRate_list[t][0])
plt.plot(cycle, s)
plt.show()

In [ ]:
# Plot saving rate of 50 agents over time
#for i in range(50):
#    cycle = []
#    s = []
#    for t in range(ThisType.T_cycle):
#        cycle.append(t)
#        s.append(SavingRate_list[t][i])
#        plt.plot(cycle,s)
        
#plt.show()

In [ ]:
ThisType.aLvlNow_hist = ThisType.aNrmNow_hist*ThisType.pLvlNow_hist
aGro41=ThisType.aLvlNow_hist[41]/ThisType.aLvlNow_hist[40]
aGro41NoU=aGro41[aGro41[:]>0.2]

In [ ]:
from matplotlib import pyplot as plt
n, bins, patches = plt.hist(aGro41NoU,50,density=True)